# 5.3 Lab: Cross-Validation and the Bootstrap

## 5.3.1 The Validation Set Approach

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd 
import math
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.regressionplots import *
from sklearn import datasets, linear_model
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from collections import OrderedDict

/home/terencevz/anaconda3/envs/jupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
Auto = pd.read_csv('https://raw.githubusercontent.com/tvanzyl/Sharing_ISL_python/master/data/Auto.csv', header=0, na_values='?')
Auto = Auto.dropna().reset_index(drop=True) # drop the observation with NA values and reindex the obs from 0
print(Auto.shape)
print(Auto.head())

(392, 9)
    mpg  cylinders  displacement  horsepower  weight  acceleration  year  \
0  18.0          8         307.0       130.0    3504          12.0    70   
1  15.0          8         350.0       165.0    3693          11.5    70   
2  18.0          8         318.0       150.0    3436          11.0    70   
3  16.0          8         304.0       150.0    3433          12.0    70   
4  17.0          8         302.0       140.0    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  


In [3]:
# split the data into training and record the index of train samples
np.random.seed(1)
train = np.random.choice(Auto.shape[0], 196, replace=False)
select = np.in1d(range(Auto.shape[0]), train)

In [4]:
# start to build the model
lm = smf.ols ('mpg~horsepower', data = Auto[select]).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.620
Model:                            OLS   Adj. R-squared:                  0.618
Method:                 Least Squares   F-statistic:                     316.4
Date:                Mon, 15 Aug 2022   Prob (F-statistic):           1.28e-42
Time:                        13:11:44   Log-Likelihood:                -592.07
No. Observations:                 196   AIC:                             1188.
Df Residuals:                     194   BIC:                             1195.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     40.3338      1.023     39.416      0.0

In [5]:
# to follow the book, get prediction for all the observations in the dataset
# here we use ~ select to exclude the result of the training samples
preds = lm.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print('-------- Test error for 1st order model --------')
print(np.mean(square_error[~select]))

-------- Test error for 1st order model --------
23.361902892587235


In [6]:
# build a model with 2nd order of features  
lm2 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0)', data = Auto[select]).fit()
preds = lm2.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print('--------Test error for 2nd order--------')
print(square_error[~select].mean())

--------Test error for 2nd order--------
20.25269085835022


In [7]:
# build a model with 3rd order of features  
lm3 = smf.ols ('mpg~horsepower + I(horsepower ** 2.0) + I(horsepower ** 3.0)', data = Auto[select]).fit()
preds = lm3.predict(Auto)
square_error = (Auto['mpg'] - preds)**2
print('--------Test rror for 3rd order--------')
print(np.mean(square_error[~select]))

""" 
These results are consistent with our previous findings: a model that predicts mpg using a quadratic function of 
horsepower performs better than a model that involves only a linear function of horsepower, 
and there is little evidence in favor of a model that uses a cubic function of horsepower.
"""

--------Test rror for 3rd order--------
20.325609365878684


' \nThese results are consistent with our previous findings: a model that predicts mpg using a quadratic function of \nhorsepower performs better than a model that involves only a linear function of horsepower, \nand there is little evidence in favor of a model that uses a cubic function of horsepower.\n'

In [8]:
# if we look at the summmary for 3rd order regression, 
# the coefficient of the 3rd order term is not statistically significant. 
# I will use this as Supporting evidence for the above claim. 
print(lm3.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.722
Model:                            OLS   Adj. R-squared:                  0.717
Method:                 Least Squares   F-statistic:                     165.9
Date:                Mon, 15 Aug 2022   Prob (F-statistic):           4.60e-53
Time:                        13:11:47   Log-Likelihood:                -561.56
No. Observations:                 196   AIC:                             1131.
Df Residuals:                     192   BIC:                             1144.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               66.5200 

## 5.3.2 Leave-One-Out Cross-Validation
The LOOCV estimates only keep one sample in the validation data and use the rest of the data to train the model. This way the training model has similar dataset comparing to the model trained on entire dataset.

In [9]:
# OLS fit 
ols_fit = smf.ols ('mpg~horsepower', data = Auto).fit()
print(ols_fit.params)

Intercept     39.935861
horsepower    -0.157845
dtype: float64


In [10]:
# GLM fit. Compare with OLS fit, the coeffs are the same
glm_fit = smf.glm('mpg~horsepower', data = Auto).fit()
print(glm_fit.params)

Intercept     39.935861
horsepower    -0.157845
dtype: float64


In [11]:
# trying CV in Python is not as easy as that in R. It will require some manual coding.
# to use some of implemented function in Python, we use Sklearn for linear model 
# from sklearn.model_selection import KFold, cross_val_score
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.linear_model import LinearRegression
# from sklearn.pipeline import Pipeline

In [12]:
# let us re-train the model in sklearn
x = pd.DataFrame(Auto.horsepower)
y = Auto.mpg

model = LinearRegression()
model.fit(x, y)
print(model.intercept_)
print(model.coef_)

39.93586102117045
[-0.15784473]


In [13]:
# loo use folds equal to # of observations. We could also choose other number of folds.
k_fold = KFold(n_splits=x.shape[0]) 
test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
print(np.mean(-test))

/home/terencevz/anaconda3/envs/jupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/terencevz/anaconda3/envs/jupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/terencevz/anaconda3/envs/jupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/terencevz/anaconda3/envs/jupyter/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this vers

24.23151351792923


In [14]:
# for higher order polynomial fit, we use pipline tool. 
# below shows how to fit an order 1 to 20 polynomial data and show the loo results
# this step may take a few mins
A = OrderedDict()
n_split = x.shape[0]
for porder in range(1, 21, 2):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=n_split) # loo use folds equal to # of observations
    test = cross_val_score(model, x, y, cv=k_fold,  scoring = 'neg_mean_squared_error', n_jobs=-1)
    A[str(porder)] = np.mean(-test)
    
print(A)

OrderedDict([('1', 24.231513517929226), ('3', 19.334984064131373), ('5', 19.03321677574123), ('7', 19.125930120970356), ('9', 19.133960633661786), ('11', 19.133770655568977), ('13', 27.7634210505149), ('15', 35.29331932316829), ('17', 43.65453200632967), ('19', 60.96443125028749)])


## 5.3.3 k-Fold Cross-Validation

In [15]:
# K-fold validation is exactly same as LOO with different n_splits parameter setup. 
# the computation time is much shorter than that of LOOCV.
np.random.seed(2)
A = OrderedDict()
n_split = 10
for porder in range(1, 21, 2):
    model = Pipeline([('poly', PolynomialFeatures(degree=porder)), ('linear', LinearRegression())])
    k_fold = KFold(n_splits=n_split) 
    test = cross_val_score(model, x, y, cv = k_fold,  scoring = 'neg_mean_squared_error', n_jobs = -1)
    A[str(porder)] = np.mean(-test)
    
print(A)

OrderedDict([('1', 27.439933652339857), ('3', 21.336606183328797), ('5', 20.905630543164584), ('7', 20.953366101158057), ('9', 21.036717405054468), ('11', 21.45456822638013), ('13', 30.810090743660375), ('15', 39.53672584543371), ('17', 48.27497388595163), ('19', 64.02376231031471)])


## 5.3.4 The Bootstrap
Bootstrap means sampling with replacement. To eliminate the effect of sample size, the norm practice is to sample the same size as original dataset with replacement.

Bootstrap can be used in a lot of other places, such as estimating the accuracy of a linear regression model coeffcients / Conduct non-parametric testing (permutation test) / Estimate some complicated probability 

In [16]:
Portfolio = pd.read_csv('https://raw.githubusercontent.com/tvanzyl/Sharing_ISL_python/master/data/Portfolio.csv', header=0)

In [17]:
# to illustrate the use of the bootstrap on this data, we must first create a function, alpha_fn(), 
# which takes as input the (X, Y) data as well as a vector indicating which observations should be used to estimate alpha.
def alpha_fn(data, index):
    X = data.X.iloc[index]
    Y = data.Y.iloc[index]
    return (np.var(Y) - np.cov(X,Y)[0,1])/(np.var(X) + np.var(Y) - 2 * np.cov(X, Y)[0,1])

In [18]:
alpha_fn(Portfolio, range(0,100))

0.5766511516104116

In [19]:
# generate one set of random index with 100 elements. The array has been sorted to show there are repeat elements.
np.sort(np.random.choice(range(0, 100), size=100, replace=True))

array([ 1,  4,  4,  7,  8,  8,  8,  9, 10, 15, 15, 16, 17, 18, 19, 20, 21,
       22, 22, 22, 26, 31, 31, 32, 33, 34, 34, 37, 38, 39, 39, 40, 40, 40,
       42, 43, 43, 43, 43, 46, 46, 47, 49, 49, 50, 50, 51, 52, 52, 55, 56,
       57, 58, 60, 61, 62, 63, 63, 66, 67, 67, 68, 68, 69, 70, 70, 70, 70,
       72, 72, 73, 74, 75, 75, 76, 76, 79, 80, 81, 82, 82, 83, 83, 84, 85,
       86, 87, 88, 90, 90, 90, 90, 90, 91, 95, 95, 96, 96, 97, 99])

In [20]:
# recall the previous function with a random set of input. 
alpha_fn(Portfolio, np.random.choice(range(0, 100), size=100, replace=True))

0.632327580798003

In [21]:
# since I am not aware of boot like function in python, I just defined an ad-hoc function called boot_python()
def boot_python(data, input_fun, iteration):
    n = Portfolio.shape[0]
    idx = np.random.randint(0, n, (iteration, n))
    stat = np.zeros(iteration)
    for i in range(len(idx)):
        stat[i] = input_fun(data, idx[i])
    
    return {'Mean': np.mean(stat), 'STD': np.std(stat)}
    

In [22]:
boot_python(Portfolio, alpha_fn, 1000)

{'Mean': 0.5811900883897445, 'STD': 0.09408713019844589}

In [23]:
# End of Chapter 5